# 06-07: Signal 与 iMessageSignal CLI 和 AppleScript iMessage 集成。

In [ ]:
// ========== 1. Signal CLI (简化示例) ==========
// Signal CLI 是一个命令行工具，可以通过子进程调用
import { spawn } from 'child_process';
class SignalClient {
  private phoneNumber: string;

  constructor(phoneNumber: string) {
    this.phoneNumber = phoneNumber;
  }

  // 发送消息
  sendMessage(recipient: string, message: string): Promise<void> {
    return new Promise((resolve, reject) => {
      const proc = spawn('signal-cli', [
        '-u', this.phoneNumber,
        'send',
        '-m', message,
        recipient
      ]);

      proc.on('close', (code) => {
        if (code === 0) {
          resolve();
        } else {
          reject(new Error(`signal-cli exited with code ${code}`));
        }
      });
    });
  }

  // 接收消息 (daemon 模式)
  receive(): Promise<string> {
    return new Promise((resolve, reject) => {
      const proc = spawn('signal-cli', [
        '-u', this.phoneNumber,
        'receive'
      ]);

      let output = '';
      proc.stdout.on('data', (data) => {
        output += data.toString();
      });

      proc.on('close', (code) => {
        if (code === 0) {
          resolve(output);
        } else {
          reject(new Error(`signal-cli exited with code ${code}`));
        }
      });
    });
  }

  // 发送群组消息
  sendGroupMessage(groupId: string, message: string): Promise<void> {
    return new Promise((resolve, reject) => {
      const proc = spawn('signal-cli', [
        '-u', this.phoneNumber,
        'send',
        '-g', groupId,
        '-m', message
      ]);

      proc.on('close', (code) => {
        if (code === 0) {
          resolve();
        } else {
          reject(new Error(`signal-cli exited with code ${code}`));
        }
      });
    });
  }
}
// 使用
// const signal = new SignalClient('+1234567890');
// signal.sendMessage('+0987654321', 'Hello from Signal!');

In [ ]:
// ========== 2. Signal 接收消息处理 ==========
interface SignalMessage {
  envelope: {
    source: string;
    sourceNumber: string;
    sourceUuid: string;
    sourceName?: string;
    sourceDevice: number;
    timestamp: number;
    receiptMessage?: {
      when: number;
      isDelivery: boolean;
      isRead: boolean;
      timestamps: number[];
    };
    dataMessage?: {
      timestamp: number;
      message?: string;
      expiresInSeconds: number;
      viewOnce: boolean;
      groupInfo?: {
        groupId: string;
        type: 'DELIVER' | 'UPDATE';
      };
    };
  };
}
class SignalDaemon {
  private handler?: (msg: SignalMessage) => void;

  onMessage(handler: (msg: SignalMessage) => void): void {
    this.handler = handler;
  }

  start(phoneNumber: string): void {
    // 实际实现需要持续监听 signal-cli daemon
    // signal-cli -u <number> daemon --system

    // 使用 JSON-RPC 接口
    const proc = spawn('signal-cli', [
      '-u', phoneNumber,
      '--output=json',
      'receive',
      '-t', '0' // 持续接收
    ]);

    proc.stdout.on('data', (data) => {
      const lines = data.toString().split('\n');
      for (const line of lines) {
        if (line.trim()) {
          try {
            const msg: SignalMessage = JSON.parse(line);
            this.handler?.(msg);
          } catch {
            // Ignore invalid JSON
          }
        }
      }
    });
  }
}
// 使用
// const daemon = new SignalDaemon();
// daemon.onMessage((msg) => {
//   if (msg.envelope.dataMessage?.message) {
//     console.log('Received:', msg.envelope.dataMessage.message);
//   }
// });
// daemon.start('+1234567890');

In [ ]:
// ========== 3. iMessage AppleScript ==========
// macOS only - 使用 AppleScript 发送 iMessage
import { exec } from 'child_process';
import { promisify } from 'util';

const execAsync = promisify(exec);
class iMessageClient {
  // 发送消息 (需要 macOS 和 Messages.app)
  async sendMessage(handle: string, message: string): Promise<void> {
    const escapedMessage = message.replace(/'/g, "\'");
    const script = `
      tell application "Messages"
        set targetService to 1st service whose service type = iMessage
        set targetBuddy to buddy "${handle}" of targetService
        send "${escapedMessage}" to targetBuddy
      end tell
    `;

    await execAsync(`osascript -e '${script}'`);
  }

  // 发送给多个收件人
  async sendToMultiple(handles: string[], message: string): Promise<void> {
    for (const handle of handles) {
      await this.sendMessage(handle, message);
    }
  }

  // 获取最近的对话
  async getRecentChats(limit: number = 10): Promise<any[]> {
    const script = `
      tell application "Messages"
        set chatList to {}
        repeat with i from 1 to ${limit}
          try
            set theChat to chat i
            set end of chatList to name of theChat
          end try
        end repeat
        return chatList
      end tell
    `;

    const { stdout } = await execAsync(`osascript -e '${script}'`);
    return stdout.split(',').map(s => s.trim());
  }
}
// 使用
// const imessage = new iMessageClient();
// imessage.sendMessage('friend@icloud.com', 'Hello from Node.js!');

In [ ]:
// ========== 4. iMessage 数据库读取 (只读) ==========
// 读取 ~/Library/Messages/chat.db (需要适当权限)
import Database from 'better-sqlite3';
import { homedir } from 'os';
import { join } from 'path';
interface ChatMessage {
  id: number;
  text: string | null;
  date: Date;
  isFromMe: boolean;
  handle: string | null;
  service: string | null; // iMessage, SMS
}
class iMessageReader {
  private db: Database.Database;

  constructor() {
    const dbPath = join(homedir(), 'Library/Messages/chat.db');
    // 需要复制数据库以避免锁定问题
    this.db = new Database(dbPath, { readonly: true });
  }

  getRecentMessages(limit: number = 50): ChatMessage[] {
    const stmt = this.db.prepare(`
      SELECT
        m.ROWID as id,
        m.text,
        datetime(m.date / 1000000000 + 978307200, 'unixepoch') as date,
        m.is_from_me as isFromMe,
        h.id as handle,
        c.service_name as service
      FROM message m
      LEFT JOIN handle h ON m.handle_id = h.ROWID
      LEFT JOIN chat_message_join cmj ON m.ROWID = cmj.message_id
      LEFT JOIN chat c ON cmj.chat_id = c.ROWID
      ORDER BY m.date DESC
      LIMIT ?
    `);

    return stmt.all(limit).map((row: any) => ({
      id: row.id,
      text: row.text,
      date: new Date(row.date),
      isFromMe: row.isFromMe === 1,
      handle: row.handle,
      service: row.service
    }));
  }

  getMessagesFromHandle(handle: string, limit: number = 50): ChatMessage[] {
    const stmt = this.db.prepare(`
      SELECT
        m.ROWID as id,
        m.text,
        datetime(m.date / 1000000000 + 978307200, 'unixepoch') as date,
        m.is_from_me as isFromMe,
        h.id as handle,
        c.service_name as service
      FROM message m
      JOIN handle h ON m.handle_id = h.ROWID
      LEFT JOIN chat_message_join cmj ON m.ROWID = cmj.message_id
      LEFT JOIN chat c ON cmj.chat_id = c.ROWID
      WHERE h.id = ?
      ORDER BY m.date DESC
      LIMIT ?
    `);

    return stmt.all(handle, limit).map((row: any) => ({
      id: row.id,
      text: row.text,
      date: new Date(row.date),
      isFromMe: row.isFromMe === 1,
      handle: row.handle,
      service: row.service
    }));
  }

  close(): void {
    this.db.close();
  }
}
// 使用
// const reader = new iMessageReader();
// const messages = reader.getRecentMessages(10);
// console.log(messages);
// reader.close();

In [ ]:
// ========== 5. 统一的跨平台消息接口 ==========
interface UnifiedMessage {
  platform: 'signal' | 'imessage' | 'telegram' | 'whatsapp';
  id: string;
  from: string;
  to: string;
  content: string;
  timestamp: Date;
  attachments?: Array<{
    type: 'image' | 'video' | 'audio' | 'file';
    url?: string;
    filename?: string;
  }>;
}
interface MessageAdapter {
  readonly platform: string;
  send(to: string, content: string): Promise<void>;
  onMessage(handler: (msg: UnifiedMessage) => void): void;
  start(): Promise<void>;
  stop(): Promise<void>;
}
class SignalAdapter implements MessageAdapter {
  readonly platform = 'signal';
  private client: SignalClient;
  private daemon?: SignalDaemon;

  constructor(phoneNumber: string) {
    this.client = new SignalClient(phoneNumber);
  }

  async send(to: string, content: string): Promise<void> {
    return this.client.sendMessage(to, content);
  }

  onMessage(handler: (msg: UnifiedMessage) => void): void {
    this.daemon = new SignalDaemon();
    this.daemon.onMessage((signalMsg) => {
      if (signalMsg.envelope.dataMessage?.message) {
        handler({
          platform: 'signal',
          id: signalMsg.envelope.sourceUuid,
          from: signalMsg.envelope.sourceNumber,
          to: 'me',
          content: signalMsg.envelope.dataMessage.message,
          timestamp: new Date(signalMsg.envelope.timestamp)
        });
      }
    });
  }

  async start(): Promise<void> {
    // signal-cli 已在构造函数中配置
  }

  async stop(): Promise<void> {
    // 清理资源
  }
}
class iMessageAdapter implements MessageAdapter {
  readonly platform = 'imessage';
  private client: iMessageClient;

  constructor() {
    this.client = new iMessageClient();
  }

  async send(to: string, content: string): Promise<void> {
    return this.client.sendMessage(to, content);
  }

  onMessage(handler: (msg: UnifiedMessage) => void): void {
    // iMessage 需要通过数据库轮询或 AppleScript 监听
    // 这里简化处理
  }

  async start(): Promise<void> {
    // macOS 环境检查
    if (process.platform !== 'darwin') {
      throw new Error('iMessage adapter requires macOS');
    }
  }

  async stop(): Promise<void> {
    // 清理资源
  }
}